In [25]:
import pandas as pd
import numpy as np

pd.__version__

'2.1.0'

In [2]:
! wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

--2023-09-18 17:19:10--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv’

housing.csv         100%[===================>]   1.36M  2.69MB/s    in 0.5s    

2023-09-18 17:19:11 (2.69 MB/s) - ‘housing.csv’ saved [1423529/1423529]



In [4]:
dataset = pd.read_csv('housing.csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [6]:
dataset.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [9]:
dataset.ocean_proximity.unique()

array(['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND'],
      dtype=object)

In [12]:
# What's the average value of the median_house_value for the houses located near the bay?
dataset.loc[dataset['ocean_proximity'] == 'NEAR BAY'].describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,2290.000000,2290.000000,2290.000000,2290.000000,2270.000000,2290.000000,2290.000000,2290.000000,2290.000000
mean,-122.260694,37.801057,37.730131,2493.589520,514.182819,1230.317467,488.616157,4.172885,259212.311790
std,0.147004,0.185434,13.070385,1830.817022,367.887605,885.899035,350.598369,2.017427,122818.537064
min,-122.590000,37.350000,2.000000,8.000000,1.000000,8.000000,1.000000,0.499900,22500.000000
25%,-122.410000,37.730000,29.000000,1431.250000,289.000000,718.250000,275.000000,2.834750,162500.000000
50%,-122.250000,37.790000,39.000000,2083.000000,423.000000,1033.500000,406.000000,3.818650,233800.000000
75%,-122.140000,37.907500,52.000000,3029.750000,628.750000,1495.000000,599.250000,5.054425,345700.000000
max,-122.010000,38.340000,52.000000,18634.000000,3226.000000,8276.000000,3589.000000,15.000100,500001.000000


### Question 6
1. Calculate the average of `total_bedrooms` column in the dataset.
2. Use the `fillna` method to fill the missing values in `total_bedrooms` with the mean value from the previous step.
3. Now, calculate the average of `total_bedrooms` again.
4. Has it changed?

In [15]:
dataset.total_bedrooms.isna().sum()

207

In [16]:
total_bedrooms_mean = dataset['total_bedrooms'].mean()
dataset['total_bedrooms'].fillna(total_bedrooms_mean)
new_total_bedrooms_mean = dataset['total_bedrooms'].mean()
print(total_bedrooms_mean == new_total_bedrooms_mean) ## sigue siendo el mismo promedio

True


### Question 7

1. Select all the options located on islands.
2. Select only columns `housing_median_age`, `total_rooms`, `total_bedrooms`.
3. Get the underlying NumPy array. Let's call it `X`.
4. Compute matrix-matrix multiplication between the transpose of `X` and `X`. To get the transpose, use `X.T`. Let's call the result `XTX`.
5. Compute the inverse of `XTX`.
6. Create an array `y` with values `[950, 1300, 800, 1000, 1300]`.
7. Multiply the inverse of `XTX` with the transpose of `X`, and then multiply the result by `y`. Call the result `w`.
8. What's the value of the last element of `w`?


In [32]:
def vector_vector_multiplication(u, v):
    assert u.shape[0] == v.shape[0]
    
    n = u.shape[0]
    
    result = 0.0

    for i in range(n):
        result = result + u[i] * v[i]
    
    return result

def matrix_vector_multiplication(U, v):
    assert U.shape[1] == v.shape[0]
    
    num_rows = U.shape[0]
    
    result = np.zeros(num_rows)
    
    for i in range(num_rows):
        result[i] = vector_vector_multiplication(U[i], v)
    
    return result

def matrix_matrix_multiplication(U, V):
    assert U.shape[1] == V.shape[0]
    
    num_rows = U.shape[0]
    num_cols = V.shape[1]
    
    result = np.zeros((num_rows, num_cols))
    
    for i in range(num_cols):
        vi = V[:, i]
        Uvi = matrix_vector_multiplication(U, vi)
        result[:, i] = Uvi
    
    return result

In [47]:
X = dataset.loc[dataset['ocean_proximity'] == 'ISLAND', ['housing_median_age','total_rooms', 'total_bedrooms']].to_numpy()
XTX = matrix_matrix_multiplication(X.T, X)
XTX_inverse = np.linalg.inv(XTX)
y = np.array([950, 1300, 800, 1000, 1300])
w = matrix_vector_multiplication(matrix_matrix_multiplication(XTX_inverse,X.T),y)
print(w)

[23.12330961 -1.48124183  5.69922946]
